In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import numpy as np
from prep_data import Dataset
import torch
from unet import UNet
from util import  Plot_model, plot_prediction




def infer1D(x,y,name=''):
	''' To infer the 1D models'''


	data  =Dataset(x,y)

	with torch.no_grad():
		# x = data.x_data.view(data.x_data.shape[0],1,-1)
		x = data.x_data
		y = data.y_data.view(data.y_data.shape[0],1,-1)
		pred = model(x)

		x = x.cpu().numpy()
		init = x[:,1,:] * 4.5
		x = x[:,0,:]
		y = y.cpu().numpy()
		y = y[:,0,:]
		pred = pred.cpu().numpy()
		pred = pred[:,0,:]



		x = x*4.5 
		y= y*4.5
		pred = pred*4.5
		plot_prediction (x,y,pred,init,5,5) # the plot save the figure
		plt.savefig(name)
		plt.close()


def infer2D(x):
	''' To infer 2D model'''

	
	x = torch.tensor(x,dtype=torch.float32).cuda()

	with torch.no_grad():
		pred = model(x)
		pred = pred.cpu().numpy()
		pred = pred[:,0,:]
		x = x.cpu().numpy()
		x = x[:,0,:]


		pred = pred*4.5

	return pred




	


In [ ]:

path= './output/'

netname = 'unet' 
feature = 16

# load nn model 
model = UNet(2,1,feature)
model.load_state_dict(torch.load(path+'NNmodel/'+netname))
model.eval().cuda()


# load training data and plot
x= np.load(path+'NNmodel/xvalid_%s.npy'%netname)
y =np.load(path+'NNmodel/yvalid_%s.npy'%netname) 
infer (x,y,'valid')


# load validation data and plot
x= np.load(path+'NNmodel/xtrain_%s.npy'%netname) 
y =np.load(path+'NNmodel/ytrain_%s.npy'%netname) 
#x0= np.load(path+'NNmodel/x0train.npy')
infer(x,y,'')




## Inferreing 2D

In [ ]:
par = {'nx':801,   'dx':0.02, 'ox':0,
       'nz':200,   'dz':0.02, 'oz':0,
       'ns':200,   'ds':0.08,   'osou':0,  'sz':0.,
       'nr':400,   'dr':0.04,  'orec':0, 'rz':0.,
       'nt':2500,  'dt':0.002,  'ot':0,

      }


#input 
flooded_m = '<path_to_flooded_model>'
inv_m = 'path_to_inverted_model'

# output
opfile='<path_to_save_unflood_model>'


 # BP model 
bp,par = np.load(inv_m)
bpf,_= np.load(flooded_m)
bp = bp.T
bpf = bpf.T

bp2 = bp.reshape((bp.shape[0],1,bp.shape[1]))
bpf = bpf.reshape((bpf.shape[0],1,bpf.shape[1]))
bp2 = np.concatenate((bp2,bpf),axis=1)

bp2 =bp2/4.5
bp2 = resize(bp2,(bp2.shape[0],bp2.shape[1],200))

print("shape of the data is ", bp.shape)
bp_unflooded = infer2D(bp2)
bp_unflooded = resize(bp_unflooded,(par['nx'],par['nz']))
Plot_model(bp.T,par,inv_m)
Plot_model(bp_unflooded.T,par,opfile)


np.save(opfile,bp_unflooded)